### Parcing data from Investing.con, selecting USA High-Volatile News. Saving data to SQL-lite

#### Imports

In [75]:
import requests
from bs4 import BeautifulSoup
import sqlite3

#### Connecting to website. Checking response

In [50]:
url = 'https://www.investing.com/economic-calendar/'
headers = {
    "Accept": "*/*",
    "User-Agent": " 
} # paste your user-agent
req = requests.get(url, headers=headers)
req

<Response [200]>

#### Selecting special data from the website

In [89]:
soup = BeautifulSoup(req.text, 'html.parser')

news = [] #Creating an empty list to store further data

for item in soup.find_all(class_="js-event-item"):
    title = item.find(class_="left event").text.strip()
    country_element = item.find(class_="left flagCur noWrap")
    country = country_element.get_text(strip=True) if country_element else ''
    
    volatility = item.find(class_="left textNum sentiment noWrap").get("title")

    time = item.get("data-event-datetime")
    link = 'https://www.investing.com' + item.find("a").get("href")
    
    previous_element = item.select_one("[class^='prev']")
    previous = previous_element.get_text(strip=True) if previous_element else 'N/A'

    forecast_element = item.select_one("[class^='fore']")
    forecast = forecast_element.get_text(strip=True) if forecast_element else 'N/A'
        

        
    if (country == "USD") and (volatility == "High Volatility Expected"):
        checking_if_everything_is_good = f'{title} {time} \nForecast: {forecast}, Previous: {previous}\nCheck more info -> {link}\n'
        print(checking_if_everything_is_good)
        news.append((title,time,forecast,previous,link))
        
#print(news)
        
        

Initial Jobless Claims 2024/02/01 08:30:00 
Forecast: 213K, Previous: 214K
Check more info -> https://www.investing.com/economic-calendar/initial-jobless-claims-294

S&P Global US Manufacturing PMI (Jan) 2024/02/01 09:45:00 
Forecast: 50.3, Previous: 47.9
Check more info -> https://www.investing.com/economic-calendar/manufacturing-pmi-829

ISM Manufacturing PMI (Jan) 2024/02/01 10:00:00 
Forecast: 47.2, Previous: 47.4
Check more info -> https://www.investing.com/economic-calendar/ism-manufacturing-pmi-173

ISM Manufacturing Prices (Jan) 2024/02/01 10:00:00 
Forecast: 46.0, Previous: 45.2
Check more info -> https://www.investing.com/economic-calendar/ism-manufacturing-prices-174



#### Creating SQL table

In [90]:
conn = sqlite3.connect('news.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE usa_news (
        title TEXT, 
        time TEXT, 
        forecast TEXT, 
        previous TEXT,
        link TEXT
    )
''')

#### Inserting data fron news list to SQL table "usa_news"

In [91]:
cursor.executemany('INSERT INTO usa_news VALUES (?, ?, ?, ?, ?)', news)

#### Checking data in the table

In [92]:
cursor = conn.cursor()
query = "SELECT * FROM usa_news"
cursor.execute(query)
rows = cursor.fetchall()
for row in rows:
    print(row)
conn.close()

('Initial Jobless Claims', '2024/02/01 08:30:00', '213K', '214K', 'https://www.investing.com/economic-calendar/initial-jobless-claims-294')
('S&P Global US Manufacturing PMI (Jan)', '2024/02/01 09:45:00', '50.3', '47.9', 'https://www.investing.com/economic-calendar/manufacturing-pmi-829')
('ISM Manufacturing PMI (Jan)', '2024/02/01 10:00:00', '47.2', '47.4', 'https://www.investing.com/economic-calendar/ism-manufacturing-pmi-173')
('ISM Manufacturing Prices (Jan)', '2024/02/01 10:00:00', '46.0', '45.2', 'https://www.investing.com/economic-calendar/ism-manufacturing-prices-174')
